In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import gurobipy as gp
from gurobipy import GRB
from gurobipy import Model, GRB
import requests
import random
import re

In [10]:
#Data Import
#Data (1-7) from GCAT (J. McDowell, planet4589.org/space/gcat), Data (8) from Our World in Data (https://ourworldindata.org/grapher/cost-space-launches-low-earth-orbit)
#1 Launch Pad
lp = pd.read_table(r'C:\Users\Luke Wolf\Desktop\Personal Projects\OR Satellite Launch Schedule\launch.tsv', header=0)
#lp.head()

#2 Satellite Catalog
sc = pd.read_table(r'C:\Users\Luke Wolf\Desktop\Personal Projects\OR Satellite Launch Schedule\satcat.tsv', header=0)
satcat = sc[sc['OpOrbit'].str.contains('Leo|LEO|LEO/I|LLEO/I', na=False)]
#print(satcat.head())

#3 Auxiliary Satellite Data
ax = pd.read_table(r'C:\Users\Luke Wolf\Desktop\Personal Projects\OR Satellite Launch Schedule\auxcat.tsv', header=0)
auxcat = ax[ax['OpOrbit'].str.contains('Leo|LEO|LEO/I|LLEO/I', na=False)]
#print(auxcat.head())

#4 Failed to orbit satellites
ftorbit = pd.read_table(r'C:\Users\Luke Wolf\Desktop\Personal Projects\OR Satellite Launch Schedule\ftocat.tsv', header=0)
#print(ftorbit)

#5 Launch List
launchlist = pd.read_table(r'C:\Users\Luke Wolf\Desktop\Personal Projects\OR Satellite Launch Schedule\launch.tsv', header=0)
launchlist = launchlist.dropna()
#launchlist.head()
 
#6 Organizations DB 
#Astronomical polities (AP), countries/regions (CY), intergovernmental organizations (IGO), academic/non-profit organizations (A), businesses (B), and civilian/military government agencies (C/D).  
orgs = pd.read_table(r'C:\Users\Luke Wolf\Desktop\Personal Projects\OR Satellite Launch Schedule\orgs.tsv', header=0)
#print(orgs.head())

#7 Sites
sites = pd.read_table(r'C:\Users\Luke Wolf\Desktop\Personal Projects\OR Satellite Launch Schedule\sites.tsv', header=0)

#8 Cost of LEO
leocost = pd.read_csv(r'C:\Users\Luke Wolf\Desktop\Personal Projects\OR Satellite Launch Schedule\cost-space-launches-low-earth-orbit.csv', header=0)
leocost['Year'] = pd.to_datetime(leocost['Year'], format='%Y')
#leocost.head()

#9 NWS Lat Lon Zone Data
nwsdata = gpd.read_file(r'C:\Users\Luke Wolf\Desktop\Personal Projects\OR Satellite Launch Schedule\z_05mr24.shp', header=0)
print(nwsdata.head())

C:\Users\Luke Wolf\AppData\Local\Temp\ipykernel_88160\3745321696.py:8: DtypeWarning: Columns (1,18,20,22,24,26,28,32,36) have mixed types. Specify dtype option on import or set low_memory=False.
  sc = pd.read_table(r'C:\Users\Luke Wolf\Desktop\Personal Projects\OR Satellite Launch Schedule\satcat.tsv', header=0)


  STATE  CWA TIME_ZONE FE_AREA ZONE     NAME STATE_ZONE      LON      LAT  \
0    AL  BMX         C      ec  019  Calhoun      AL019 -85.8261  33.7714   
1    AL  MOB         C      sc  057   Butler      AL057 -86.6803  31.7524   
2    AL  BMX         C      se  046  Bullock      AL046 -85.7161  32.1005   
3    AL  BMX         C      cc  017   Blount      AL017 -86.5674  33.9809   
4    AL  BMX         C      cc  034     Bibb      AL034 -87.1264  32.9986   

  SHORTNAME                                           geometry  
0   Calhoun  POLYGON ((-85.53010 33.94181, -85.53200 33.897...  
1    Butler  POLYGON ((-86.44800 31.96431, -86.44769 31.952...  
2   Bullock  POLYGON ((-85.43349 32.23421, -85.43120 32.198...  
3    Blount  POLYGON ((-86.45229 34.25951, -86.44420 34.257...  
4      Bibb  POLYGON ((-87.02679 33.24601, -87.02599 33.231...  


In [11]:
#Data Cleaning and Manipulation
#Cost Average from last 20 years Per Launch Class
leoyear = leocost[leocost['Year'] >= '2000-01-01']
leoavgcost = leoyear.groupby('launch_class')['cost_per_kg'].mean()
leoavgcost = pd.DataFrame(leoavgcost).reset_index()
#print(leoavgcost)
#Sites only located in the Southeast US Region (Florida, Georgia, Alabama, Mississippi, Louisiana, Texas)
locations = 'Florida|Georgia|Alabama|Mississippi|Louisiana|Texas|FL|GA|AL|MS|LA|TX|US|United States'
filtered_sites = sites[sites['Location'].str.contains(locations, na=False)]
filtered_sites = filtered_sites.reset_index(drop=True)
#filtered_sites['State, Country'] = filtered_sites['Location'].str.split(',').str[1] + ", " + filtered_sites['StateCode']
filtered_sites['State'] = filtered_sites['Location'].str.contains(locations, na=False)
'''
def state_short(state):
    state_str = str(state)
    if re.search(r'Florida|FL', state_str):
        return 'Fl'
    elif re.search(r'Georgia|GA', state_str):
        return 'Ga'
    elif re.search(r'Alabama|AL', state_str):
        return 'Al'
    elif re.search(r'Mississippi|MS', state_str):
        return 'Ms'
    elif re.search(r'Louisiana|LA', state_str):
        return 'La'
    elif re.search(r'Texas|TX', state_str):
        return 'Tx'
    else:
        return 'US'

filtered_sites['State'] = filtered_sites['State'].apply(state_short)
'''
print(filtered_sites.head())
print(len(filtered_sites))
#filtered_sites.to_excel(r'C:\Users\Luke Wolf\Desktop\Personal Projects\OR Satellite Launch Schedule\filtered_sites.xlsx', index=False)

#Combining Satellite Catalog (satcat, auxcat, ftorbit)
df = pd.concat([satcat, auxcat, ftorbit], axis=0, ignore_index=True)
print(df.head())
df.to_excel(r'C:\Users\Luke Wolf\Desktop\Personal Projects\OR Satellite Launch Schedule\Satellite Launches.xlsx', index=False)
#Cost Assumptions of Satellites (Auxcat, Satcat, Ftorbit) with Orgaization Type
'''
Cost value is arbitary since  there is no cost data for these satellites
Additionally, the real cost does not matter since the OR solver will optimize the cost (no matter the value)
'''
df['TotMass'] = df['TotMass'].replace({'?', '-'}, 0)
df['TotMass'] = df['TotMass'].astype(float)

lq = df['TotMass'].quantile(0.25)
uq = df['TotMass'].quantile(0.75)
mean = np.mean(df['TotMass'])
print(lq, uq, mean)

def random_cost(row):
    if row['TotMass'] < 260: 
        return leoavgcost['cost_per_kg'][1]
    elif row['TotMass'] >= 260: #Upper Quartile of Total Mass (right skweded dis.)
        return leoavgcost['cost_per_kg'][2]
    else:
        return leoavgcost['cost_per_kg'][0]

df['Cost'] = df.apply(random_cost, axis=1)
#Final Dataframe
df = df.drop(columns=['PF', 'AF', 'IF', 'MassFlag', 'OQUAL', 'LFlag', 'DryFlag', 'DFlag', 'SpanFlag'])
df['State'] = df['State'].str.contains(locations, na=False)
dataframe = pd.merge(df, filtered_sites, on='State', how='right')
print(dataframe.head())
#make an excel file with the data
#dataframe.to_excel(r'C:\Users\Luke Wolf\Desktop\Personal Projects\OR Satellite Launch Schedule\df.xlsx', index=False)

#Creating Weather Probability for Southeast US Regaion from National Weather Service (NWS) API
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points 
    on the Earth surface.
    :param lat1, lon1: Latitude and Longitude of point 1 (in decimal degrees)
    :param lat2, lon2: Latitude and Longitude of point 2 (in decimal degrees)
    :return: Distance in kilometers between the two points.
    """
    # convert decimal degrees to radians 
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # haversine formula 
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371 # Radius of earth in kilometers
    return c * r

def find_closest_zone(dataframe, nwsdata):
    min_distances = []
    closest_zones = []
    for index, row in dataframe.iterrows():
        distances = nwsdata.apply(lambda x: haversine(row['lat'], row['lon'], x['LAT'], x['LON']), axis=1)
        min_distance_index = distances.idxmin()
        closest_zone = nwsdata.iloc[min_distance_index]['ZONE']
        min_distances.append(distances[min_distance_index])
        closest_zones.append(closest_zone)
    dataframe['closest_zone'] = closest_zones
    dataframe['distance_to_closest_zone'] = min_distances
    return dataframe

dataframe = find_closest_zone(dataframe, nwsdata)
weatherdf = []

for index, row in filtered_sites.iterrows():
    zone = row['closest_zone']
    url = 'https://api.weather.gov/zones/forecast/{zone}/observations'
    print(f'{url} {row["Location"]}')

    response = requests.get(url)
    if response.status_code == 200:
        forecast_data = response.json()
        
        for period in forecast_data['properties']['periods']:
            print(f"{period['name']}: {period['detailedForecast']}")
    else:
        print(f"Failed to fetch forecast for {row['Location']}")
print(forecast_data)

    #Site Code UCode Type StateCode       TStart        TStop ShortName  \
0   AFMTC    -    CC   LS        US  1951 Jun 30  1955 Dec 16     AFMTC   
1    BARK    -  BARK   LS        US            -            -   BARK  -   
2     CCA    -   CCA   LS        US         1984  1998 Jan  1    CC (A)   
3   CCAFS    -    CC   LS        US     1974 Mar         1992     CCAFS   
4  CCAFS1    -    CC   LS        US  2000 Feb  4  2020 Dec  9     CCAFS   

                                                Name                 Location  \
0                      Air Force Missile Test Center  Cape Canaveral, Florida   
1                                      Barksdale AFB                Louisiana   
2  Cape Canaveral Air Force Station, Eastern Test...  Cape Canaveral, Florida   
3  Cape Canaveral Air Force Station, Eastern Test...  Cape Canaveral, Florida   
4  Cape Canaveral Air Force Station, Eastern Test...  Cape Canaveral, Florida   

       Longitude     Latitude  Error Parent ShortEName EName G

KeyError: 'lat'

In [ ]:
#Data Visualization Prior to Modeling
'''
plt.figure(figsize=(20, 20))
sns.set_style('darkgrid')
sns.set_context('talk')
sns.pairplot(df, hue='OpOrbit', palette='viridis')
'''

In [ ]:
#Optimization Modeling: Cost Minimization of LEO Satellite Launches
model = Model("LEO Satellite Optimization")
#Decision Variables
Xij = 1 if satellite i is launched from launch site j
Xij = 0 otherwise

#Objective Function
#Minimize the cost of a satellite 